In [ ]:
## Import different packages to be used
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from fooof import FOOOFGroup
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from statsmodels.stats.anova import AnovaRM
import pingouin as pg
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
## Extract eeg data and somno scoring from each subject of interest

# Function to load EEG and somno scoring CSV files for a given subject
def load_data(subject_id):
    eeg_data = pd.read_csv(input(f"Enter location of CSV file for {subject_id} which includes EEG for somnotate scoring: "))
    somno_scoring = pd.read_csv(input(f"Enter location of file for {subject_id} of the somnotate scoring converted back to CSV: "))
    return eeg_data, somno_scoring

# Load data for subjects
eeg_data_sub_007, somno_scoring_sub_007 = load_data("sub-007")
eeg_data_sub_010, somno_scoring_sub_010 = load_data("sub-010")
eeg_data_sub_011, somno_scoring_sub_011 = load_data("sub-011")
eeg_data_sub_015, somno_scoring_sub_015 = load_data("sub-015")
eeg_data_sub_016, somno_scoring_sub_016 = load_data("sub-016")
eeg_data_sub_017, somno_scoring_sub_017 = load_data("sub-017")

In [ ]:
## Save the EEG data and somno scoring as pickle files

eeg_data_sub_007.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/eeg_data_sub-007_ses-01_recording-01.pkl")
print("eeg_data_sub-007 saved as pickle file")
somno_scoring_sub_007.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-007_ses-01_recording-01.pkl")
print("somno_scoring_sub-007 saved as pickle file")
eeg_data_sub_010.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno//eeg_data_sub-010_ses-01_recording-01.pkl")
print("eeg_data_sub-010 saved as pickle file")
somno_scoring_sub_010.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-010_ses-01_recording-01.pkl")
print("somno_scoring_sub-010 saved as pickle file")
eeg_data_sub_011.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/eeg_data_sub-011_ses-01_recording-01.pkl")
print("eeg_data_sub-011 saved as pickle file")
somno_scoring_sub_011.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-011_ses-01_recording-01.pkl")
print("somno_scoring_sub-011 saved as pickle file")
eeg_data_sub_015.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/eeg_data_sub-015_ses-01_recording-01.pkl")
print("eeg_data_sub-015 saved as pickle file")
somno_scoring_sub_015.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-015_ses-01_recording-01.pkl")
print("somno_scoring_sub-015 saved as pickle file")
eeg_data_sub_016.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/eeg_data_sub-016_ses-02_recording-01.pkl")
print("eeg_data_sub-016 saved as pickle file")
somno_scoring_sub_016.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-016_ses-02_recording-01.pkl")
print("somno_scoring_sub-016 saved as pickle file")
eeg_data_sub_017.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/eeg_data_sub-017_ses-01_recording-01.pkl")
print("eeg_data_sub-017 saved as pickle file")
somno_scoring_sub_017.to_pickle("Z:/somnotate/to_score_set/pickle_eeg_somno/somno_scoring_sub-017_ses-01_recording-01.pkl")
print("somno_scoring_sub-017 saved as pickle file")

In [ ]:
## Parameters

# Set sampling rate
fs = int(input("Sampling rate of signal in Hz: "))

# State duration for exponent analysis
epoch_duration = int(input("Enter length of epoch for exponent analysis in seconds: "))


In [ ]:
## Use neurodsp to determine the PSD for both channels at the epoch of interest in each subject

# Define a small number (epsilon) to add to 0 values
epsilon = 1e-10

# Specify the channels of interest
channels_of_interest = ['EEG1', 'EEG2']

# Function to calculate the PSD for each subject, focusing on specific channels
def calculate_psd(eeg_data, fs, epoch_duration, target_channels):
    # Prepare a dictionary to store PSD values and frequency data for each channel
    psd_values_dict = {}
    frequencies = None  # Frequencies are the same for each bin

    # Loop over each EEG channel, filtered by target channels
    for channel, sig in eeg_data.items():
        if channel not in target_channels:
            continue  # Skip channels not in the list
        
        # Initialize a list to store PSD values for the current channel
        psd_values_dict[channel] = []

        # Calculate total recording time in seconds and number of bins
        recording_seconds = len(sig) / fs  # Assuming all channels have the same length
        num_bins = int(recording_seconds // epoch_duration)  # For the entirety of recording length
        samples_per_bin = fs * epoch_duration

        # Loop over each bin
        for i in range(num_bins):
            # Extract the data for the current bin
            start = i * samples_per_bin
            end = start + samples_per_bin
            bin_data = sig[start:end]
            
            # Compute the power spectrum for this bin using Welch's method
            freqs, psd = compute_spectrum(bin_data, fs, method='welch', avg_type='mean', nperseg=fs*2)
            
            # Store the PSD values
            psd_values_dict[channel].append(psd)
            
            # Store frequencies once (frequencies are the same for all bins)
            if frequencies is None:
                frequencies = freqs

    # Convert psd_values to NumPy arrays for easier handling
    for channel in psd_values_dict:
        psd_values_dict[channel] = np.array(psd_values_dict[channel])
        
        # Replace zeros with epsilon for all PSD values
        psd_values_dict[channel][psd_values_dict[channel] == 0] = epsilon

    return psd_values_dict, frequencies

# Assume we have EEG data for multiple subjects
subjects_data = {
    'sub-007': eeg_data_sub_007,
    'sub-010': eeg_data_sub_010,
    'sub-011': eeg_data_sub_011,
    'sub-015': eeg_data_sub_015,
    'sub-016': eeg_data_sub_016,
    'sub-017': eeg_data_sub_017
}

all_psd_values = {}

# Loop over each subject's EEG data and calculate the PSD
for subject_id, eeg_data in subjects_data.items():
    psd_values, frequencies = calculate_psd(eeg_data, fs, epoch_duration)
    all_psd_values[subject_id] = psd_values

# Example of how to access the PSD for a particular subject and channel
psd_values_sub_010_eeg1 = all_psd_values['sub-010']['EEG1']
psd_values_sub_010_eeg2 = all_psd_values['sub-010']['EEG2']


In [ ]:
## Run all channels and all subjects through FOOOF modelling

# Parameters for FOOOF fitting
fooof_params = {
    'peak_width_limits': [1.0, 8.0],
    'max_n_peaks': 6,
    'min_peak_height': 0.1,
    'peak_threshold': 2.0,
    'aperiodic_mode': 'fixed'
}

# Initialize a dictionary to store FOOOF results for all subjects
fooof_results = {}

# Loop over each subject's PSD values
for subject_id, psd_values in all_psd_values.items():
    fooof_results[subject_id] = {}  # Create an entry for the subject
    
    # Loop over each channel's PSD values
    for channel, psd_matrix in psd_values.items():
        # Skip EMG channel and sleepStage column
        if channel in ['EMG', 'sleepStage']:
            print(f"Skipping {channel} for subject {subject_id}")
            continue
        
        # Initialize a FOOOFGroup object with specified parameters
        fg = FOOOFGroup(**fooof_params)
        
        # Fit FOOOF model across the matrix of power spectra for this channel
        fg.fit(frequencies, psd_matrix, [2, 40])
        
        # Store the FOOOFGroup object for this channel
        fooof_results[subject_id][channel] = fg

# Example: Accessing results
# To get the FOOOF results for sub-007, EEG1:
fg_sub_007_eeg1 = fooof_results['sub-007']['EEG1']

# Print or save the FOOOF results for all subjects and channels
for subject_id, channel_results in fooof_results.items():
    print(f"Subject: {subject_id}")
    for channel, fg in channel_results.items():
        print(f"  Channel: {channel}")
        print(f"    {fg.get_results()}")

In [ ]:
## Save the FOOOF model fitting results as pickle files



In [ ]:
## Get the average aperiodic component for each of the subjects of interest

# Initialize a dictionary to store average exponents for each subject
avg_exps_all_subjects = {}

# Loop over each subject's FOOOF results
for subject_id, channel_results in fooof_results.items():
    exponents = []  # List to collect exponents from all channels for this subject
    
    for channel, fg in channel_results.items():
        # Extract the aperiodic exponent for the current channel
        exps = fg.get_params('aperiodic_params', 'exponent')
        exponents.append(exps)
    
    # Calculate the average exponent across channels
    avg_exps_all_subjects[subject_id] = np.mean(exponents, axis=0)

# Example: Accessing avg_exps for a specific subject
avg_exps_sub_007 = avg_exps_all_subjects['sub-007']
avg_exps_sub_010 = avg_exps_all_subjects['sub-010']
avg_exps_sub_011 = avg_exps_all_subjects['sub-011']

# Print the results for verification
for subject_id, avg_exps in avg_exps_all_subjects.items():
    print(f"Subject: {subject_id}, Average Exponent: {avg_exps}")


In [ ]:
## Convert somno scoring from each subject into the required epoch period to match exponent

# List of subjects and their corresponding somno_scoring DataFrames
subjects = {
    "sub-007": somno_scoring_sub_007,
    "sub-010": somno_scoring_sub_010,
    "sub-011": somno_scoring_sub_011,
}

# Dictionary to store processed somno scoring for each subject
processed_somno_scoring = {}

# Loop through each subject
for subject_id, somno_df in subjects.items():
    
    # Calculate total recording seconds
    recording_seconds = len(somno_df) / fs
   
    # Ensure the Timestamp column is in datetime format
    somno_df["Timestamp"] = pd.to_datetime(somno_df["Timestamp"])

    # Define the expected range of timestamps (1 Hz for the entire duration)
    start_time_data = somno_df["Timestamp"].iloc[0]
    end_time = start_time_data + pd.Timedelta(seconds=int(recording_seconds)) - pd.Timedelta(seconds=1)
    expected_timestamps = pd.date_range(start=start_time_data, end=end_time, freq="1S")

    # Convert timestamps to seconds relative to the first timestamp
    somno_df["second"] = (somno_df["Timestamp"] - start_time_data).dt.total_seconds().astype(int)

    # Downsample by taking the mode of sleepStage and the first timestamp in each second
    downsampled_df = somno_df.groupby("second").agg({
        "Timestamp": "first",  # Take the first timestamp in each group
        "sleepStage": lambda x: x.mode().iloc[0] if not x.mode().empty else None  # Mode for sleepStage
    }).reset_index(drop=True)

    # Round the Timestamp column to the nearest second
    downsampled_df["Timestamp"] = downsampled_df["Timestamp"].dt.round("S")

    # Reindex to align with the expected timestamps
    downsampled_df = downsampled_df.set_index("Timestamp")
    downsampled_df = downsampled_df.reindex(expected_timestamps)
    downsampled_df = downsampled_df.reset_index().rename(columns={"index": "Timestamp"})

    # Fill in missing sleepStage values if necessary
    downsampled_df["sleepStage"] = downsampled_df["sleepStage"].fillna(method="ffill").fillna(method="bfill")
    downsampled_df["sleepStage"] = downsampled_df["sleepStage"].astype(int)

    # Convert to epoch_duration
    grouped = (
        downsampled_df.groupby(downsampled_df.index // epoch_duration)
        .agg({
            "Timestamp": "first",  # Take the value at the start of the epoch for Timestamp
            "sleepStage": lambda x: x.mode().iloc[0] if not x.mode().empty else None  # Mode for sleepStage
        })
    )

    # Reset index for a clean output
    subset_df = grouped.reset_index(drop=True)
    subset_df["Timestamp"] = subset_df["Timestamp"].dt.round("10S")

    # Store the processed DataFrame for the current subject
    processed_somno_scoring[subject_id] = subset_df

    # Display results for each subject
    print(f"Processed somno scoring for {subject_id}:")
    print(subset_df.head())



In [ ]:
## Add the avg_exps from each subject as an extra column to the processed somno scoring

# Ensure avg_exps is stored in a dictionary with the corresponding subject IDs
avg_exps_dict = {
    "sub-007": avg_exps_sub_007,
    "sub-010": avg_exps_sub_010,
    "sub-011": avg_exps_sub_011,
}

# Loop through each subject
for subject_id, subset_df in processed_somno_scoring.items():
    # Retrieve the avg_exps array for the subject
    avg_exps = avg_exps_dict[subject_id]
    
    # Check if lengths match
    if len(avg_exps) == len(subset_df):
        # Add avg_exps as a new column
        subset_df["avg_exps"] = avg_exps
        print(f"Added avg_exps to {subject_id} processed somno scoring.")
    else:
        print(f"Length mismatch for {subject_id}: avg_exps ({len(avg_exps)}) vs subset_df ({len(subset_df)})")

    # Save the updated DataFrame back to the dictionary
    processed_somno_scoring[subject_id] = subset_df

# Verify the updated processed_somno_scoring
for subject_id, subset_df in processed_somno_scoring.items():
    print(f"Processed somno scoring for {subject_id}:")
    print(subset_df.head())


In [ ]:
## Create a combined df for visualising and stats for all subjects and stages

# Add the subject name to each individual dataframe before concatenating
processed_somno_scoring["sub-007"]['subject'] = 'sub-007'
processed_somno_scoring["sub-010"]['subject'] = 'sub-010'
processed_somno_scoring["sub-011"]['subject'] = 'sub-011'

# Combine processed somno scoring data from all subjects into a single DataFrame
combined_df = pd.concat(
    [processed_somno_scoring["sub-007"], processed_somno_scoring["sub-010"], processed_somno_scoring["sub-011"]],
    ignore_index=True
)


In [ ]:
## Optional choice for removing negative exponent values

# Remove rows where avg_exps is less than 0
filtered_combined_df = combined_df[combined_df['avg_exps'] >= 0]

# Find and print rows where avg_exps was less than 0
invalid_rows = combined_df[combined_df['avg_exps'] < 0]
print("The following avg_exp values were below zero and removed for plotting and group comparisons")
for index, row in invalid_rows.iterrows():
    print(f"Subject: {row['subject']}, avg_exps: {row['avg_exps']}")

In [ ]:
## Insert ZT column into combined df

def add_zt_column(df, timestamp_col="Timestamp", reference_time="09:00:00"):
    """
    Add a Zeitgeber Time (ZT) column to a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame containing a timestamp column.
    - timestamp_col (str): Name of the timestamp column. Default is "Timestamp".
    - reference_time (str): Time of day (HH:MM:SS) to use as ZT=0. Default is "09:00:00".
    
    Returns:
    - pd.DataFrame: DataFrame with an added "ZT" column.
    """
    # Ensure the timestamp column is in datetime format
    df = df.copy()  # Work on a copy to avoid modifying the original DataFrame
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Define ZT=0 as reference_time on the first day
    start_zt = pd.to_datetime(f"{df[timestamp_col].iloc[0].date()} {reference_time}")
    
    # Calculate Zeitgeber Time (ZT)
    df["ZT"] = (df[timestamp_col] - start_zt).dt.total_seconds() / 3600  # Convert seconds to hours
    
    # Adjust ZT to be between 0 and 24 hours
    df["ZT"] = df["ZT"] % 24
    
    return df

combined_df_zt = add_zt_column(combined_df)


In [ ]:
## Plot all exponent value of each subject in each sleep stage of filtered exponents which have excluded negative values

# Ensure the sleepStage column is of categorical type
filtered_combined_df['sleepStage'] = filtered_combined_df['sleepStage'].astype('category')

# Assuming filtered_combined_df is already available
# Convert sleepStage to a numeric type to allow addition
filtered_combined_df['sleepStage_numeric'] = filtered_combined_df['sleepStage'].cat.codes

# Apply jitter to the numeric sleepStage values to spread out the markers horizontally
jitter_strength = 0.3  # Controls how much to spread out the points
filtered_combined_df['sleepStage_jittered'] = (
    filtered_combined_df['sleepStage_numeric'] 
    + np.random.uniform(-jitter_strength, jitter_strength, size=len(filtered_combined_df))
)

# Create the scatter plot
plt.figure(figsize=(10, 6))

# Scatter plot with 'sleepStage_jittered' on the x-axis, 'avg_exps' on the y-axis, and 'subject' as hue
sns.scatterplot(
    data=filtered_combined_df, 
    x='sleepStage_jittered', 
    y='avg_exps', 
    hue='subject', 
    palette='Set1', 
    s=50,           # Adjust marker size to make them smaller
    marker='o', 
    edgecolor='black', 
    alpha=0.5       # Make markers more transparent
)

# Add labels and title
plt.ylabel('Exponent Value (a.u.)', fontsize=14)
plt.xlabel('')  # Remove x-axis label
plt.title('1/f Exponent by Sleep Stage', fontsize=16)

# Customize x-ticks to be 1, 2, and 3 for sleep stages
plt.xticks([0, 1, 2], ['Awake', 'NREM', 'REM'])

# Remove top and right spines
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show legend for subjects with adjusted placement
plt.legend(title="Subjects", loc='upper right', fontsize=10)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
## Plot all exponent value in an epoch of each subject in each sleep stage which are not filtered for negative values

# Identify continuous sleepStage bouts
combined_df_zt['bout_group'] = (combined_df_zt['sleepStage'] != combined_df_zt['sleepStage'].shift()).cumsum()

# Group by sleepStage and bout_group, and calculate average exponent value for each bout
bout_avg_exp = (
    combined_df_zt.groupby(['subject', 'sleepStage', 'bout_group'])['avg_exps']
    .mean()
    .reset_index()
)

# Map sleepStage to readable labels
stage_labels = {1: 'Awake', 2: 'NREM', 3: 'REM'}
bout_avg_exp['stage'] = bout_avg_exp['sleepStage'].map(stage_labels)

# Calculate the mean for each stage
stage_means = bout_avg_exp.groupby('stage')['avg_exps'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.stripplot(
    data=bout_avg_exp,
    x='stage',
    y='avg_exps',
    hue='subject',  # Different colors for each subject
    palette='Set1',
    dodge=True,  # To separate points slightly if multiple subjects share the same stage
    alpha=0.5,
    size=5,  # Smaller point size for better visibility
    jitter=0.5,  # Randomly jitter points to avoid exact overlap
)

# Add horizontal lines for the mean exponent value for each stage
for i, row in stage_means.iterrows():
    plt.hlines(y=row['avg_exps'], xmin=i-0.5, xmax=i+0.5, colors='k', linewidth=5)

# Customize the plot
plt.xlabel('')
plt.ylabel('Exponent Value (a.u.)')
plt.title('1/f Exponent in Bout Across Sleep Stages ')
plt.legend(title='Subject')
plt.grid(axis='y', alpha=0.3)

# Remove top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

# Run ANOVA comparing exponent values across sleep stages
# Extract exponent values for each stage
awake_data = bout_avg_exp[bout_avg_exp['stage'] == 'Awake']['avg_exps']
nrem_data = bout_avg_exp[bout_avg_exp['stage'] == 'NREM']['avg_exps']
rem_data = bout_avg_exp[bout_avg_exp['stage'] == 'REM']['avg_exps']

# Perform one-way ANOVA
f_stat, p_value = stats.f_oneway(awake_data, nrem_data, rem_data)

# If ANOVA is significant, run Tukey's HSD for post-hoc analysis
if p_value < 0.05:
    print("There is a significant difference in exponent values across sleep stages. Performing Tukey's HSD post-hoc test...")

    # Prepare the data for Tukey's HSD test
    bout_avg_exp['stage'] = bout_avg_exp['stage'].astype('category')  # Make stage a categorical variable for Tukey's HSD
    tukey_result = pairwise_tukeyhsd(
        bout_avg_exp['avg_exps'],  # Dependent variable
        bout_avg_exp['stage'],     # Independent variable (sleep stage)
        alpha=0.05                # Significance level
    )

    # Display Tukey's HSD result
    print(tukey_result.summary())
else:
    print("There is no significant difference in exponent values across sleep stages.")

In [ ]:
## Calculate mean/error and complete anova comparisons when combining all subjects

# Group by sleepStage and calculate mean and standard error for avg_exps
summary_stats = combined_df.groupby("sleepStage")["avg_exps"].agg(
    mean="mean",
    sem=lambda x: np.std(x, ddof=1) / np.sqrt(len(x))  # Standard error
).reset_index()

# Display the summary statistics
print(summary_stats)

In [ ]:

# Ensure the combined dataframe has sleepStage and avg_exps columns
# Perform ANOVA across the three sleep stages
anova_results = stats.f_oneway(
    combined_df[combined_df["sleepStage"] == 1]["avg_exps"],  # Stage 1 data
    combined_df[combined_df["sleepStage"] == 2]["avg_exps"],  # Stage 2 data
    combined_df[combined_df["sleepStage"] == 3]["avg_exps"]   # Stage 3 data
)

# Print the results of the ANOVA
print("ANOVA Results:")
print(f"F-statistic: {anova_results.statistic}")
print(f"P-value: {anova_results.pvalue}")


In [ ]:
# Assuming combined_df has 'sleepStage' and 'avg_exps' columns with data from all subjects

# Perform Tukey's HSD test for multiple comparisons
tukey_results = pairwise_tukeyhsd(
    combined_df['avg_exps'],  # The dependent variable
    combined_df['sleepStage'],  # The independent variable (grouping variable)
    alpha=0.05  # Significance level
)

# Print the results of Tukey's HSD test
print(tukey_results.summary())

# If you want to extract the specific results into a DataFrame for easier interpretation
tukey_df = pd.DataFrame(data=tukey_results.summary().data[1:], columns=tukey_results.summary().data[0])
print(tukey_df)

In [ ]:
## Plot boxplot running ANOVA and plotting individual significant comparisons

# Perform one-way ANOVA
sleep_stages = combined_df_zt['sleepStage'].unique()
avg_exps_by_stage = [combined_df_zt.loc[combined_df_zt['sleepStage'] == stage, 'avg_exps'] for stage in sleep_stages]

# Perform one-way ANOVA
f_value, p_value = f_oneway(*avg_exps_by_stage)

print(f'One-way ANOVA results: F={f_value:.4f}, p={p_value:.4f}')

# Create the boxplot
plt.figure(figsize=(8, 6))

# Define lighter versions of red, blue, and green
box_colors = ['lightcoral', 'lightskyblue', 'lightgreen']  # Lighter color options

# Define properties for outliers (make them more transparent)
flierprops = dict(marker='o', color='gray', alpha=0.4, markersize=5)

# Create the boxplot with custom colors
sns.boxplot(
    data=combined_df_zt, 
    x='sleepStage', 
    y='avg_exps', 
    palette=box_colors,  # Use the custom color palette
    showfliers=True, 
    width=0.5,  # Adjust box width to make them more compact
    linewidth=2.5,
    flierprops=flierprops
)

# Add labels and title
plt.ylabel('Exponent Value (a.u.)', fontsize=14)
plt.xlabel('')  # Remove x-axis label
plt.title('1/f Exponent in each Epoch Across Sleep Stage', fontsize=16)

# Customize x-ticks to be 1, 2, and 3 for sleep stages
plt.xticks([0, 1, 2], ['Awake', 'NREM', 'REM'])

# Perform post-hoc pairwise comparisons using Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=combined_df_zt['avg_exps'], groups=combined_df_zt['sleepStage'], alpha=0.05)

# Plot significant differences
y_offset = 0.5  # Initialize y-offset
for row in tukey._results_table.data[1:]:  # Skip the header row
    p_value = float(row[3])  # Get p-value
    group1, group2 = row[0], row[1]
    x1 = sleep_stages.tolist().index(group1)
    x2 = sleep_stages.tolist().index(group2)
    y = np.max(combined_df_zt['avg_exps']) + y_offset  # y-coordinate for the asterisk
    
    # Determine number of asterisks based on p-value
    if p_value < 0.001:
        asterisks = '***'
    elif p_value < 0.01:
        asterisks = '**'
    elif p_value < 0.05:
        asterisks = '*'
    else:
        asterisks = ''
    
    plt.plot([x1, x2], [y, y], 'k-', lw=1)  # Line connecting the two groups
    plt.text((x1 + x2) / 2, y, asterisks, ha='center', va='bottom', fontsize=14)
    
    y_offset -= 0.2  # Decrease y-offset for each comparison

# Remove top and right spines
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.tight_layout()
plt.savefig(f"Z:/volkan/fooof/figures/stage_comparison/all_sub_boxplot_ANOVA.png")

In [ ]:
## Plot all exponent value in an epoch of each subject in each sleep stage which are not filtered for negative values - one-way ANOVA and comparisons

# Identify continuous sleepStage bouts
combined_df_zt['bout_group'] = (combined_df_zt['sleepStage'] != combined_df_zt['sleepStage'].shift()).cumsum()

# Group by sleepStage and bout_group, and calculate average exponent value for each bout
bout_avg_exp = (
    combined_df_zt.groupby(['subject', 'sleepStage', 'bout_group'])['avg_exps']
    .mean()
    .reset_index()
)

# Map sleepStage to readable labels
stage_labels = {1: 'Awake', 2: 'NREM', 3: 'REM'}
bout_avg_exp['stage'] = bout_avg_exp['sleepStage'].map(stage_labels)

# Calculate the mean for each stage
stage_means = bout_avg_exp.groupby('stage')['avg_exps'].mean().reset_index()

# Perform one-way ANOVA using scipy's f_oneway
anova_result = stats.f_oneway(
    bout_avg_exp[bout_avg_exp['stage'] == 'Awake']['avg_exps'],
    bout_avg_exp[bout_avg_exp['stage'] == 'NREM']['avg_exps'],
    bout_avg_exp[bout_avg_exp['stage'] == 'REM']['avg_exps']
)

# Print ANOVA results
print("ANOVA Results:")
print(f"F-statistic: {anova_result.statistic:.3f}")
print(f"p-value: {anova_result.pvalue:.3e}")

# Check if ANOVA is significant
if anova_result.pvalue < 0.05:  # If p-value is less than 0.05, perform Tukey HSD
    # Perform Tukey HSD post-hoc test
    tukey_results = pairwise_tukeyhsd(endog=bout_avg_exp['avg_exps'], groups=bout_avg_exp['stage'], alpha=0.05)
    
    # Print Tukey HSD results
    print("\nTukey HSD Results:")
    print(tukey_results.summary())
    
    # Create a dictionary to store significance
    significance_dict = {}
    for comparison in tukey_results.summary().data[1:]:
        group1, group2 = comparison[0], comparison[1]
        p_value = comparison[3]
        if p_value < 0.001:
            significance_dict[(group1, group2)] = '***'
        elif p_value < 0.01:
            significance_dict[(group1, group2)] = '**'
        elif p_value < 0.05:
            significance_dict[(group1, group2)] = '*'
else:
    significance_dict = {}

# Plotting
plt.figure(figsize=(12, 6))

# Overlay the stripplot to show individual points
sns.stripplot(
    data=bout_avg_exp,
    x='stage',
    y='avg_exps',
    hue='subject',  # Different colors for each subject
    palette='Set1',
    dodge=True,  # Separate points slightly if multiple subjects share the same stage
    alpha=0.3,
    size=5,  # Adjust dot size for better visibility
    jitter=0.4,  # Adjust jitter for variation in horizontal positioning
)

# Add X markers at the mean exponent value for each stage
stage_means = bout_avg_exp.groupby('stage')['avg_exps'].mean().reset_index()
for i, row in stage_means.iterrows():
    # Plot "X" markers at the mean value for each stage
    plt.scatter(x=i, y=row['avg_exps'], color='k', marker='x', s=100)

# Add significance asterisks between groups based on Tukey HSD
for (group1, group2), significance in significance_dict.items():
    x1 = list(stage_labels.values()).index(group1)
    x2 = list(stage_labels.values()).index(group2)
    y_max = max(bout_avg_exp[bout_avg_exp['stage'] == group1]['avg_exps'].max(), 
                bout_avg_exp[bout_avg_exp['stage'] == group2]['avg_exps'].max()) + 0.1
    # Plot the bar (line) between the two groups
    plt.plot([x1, x2], [y_max, y_max], color='k', lw=1.5)
    plt.text((x1 + x2) / 2, y_max, significance, ha='center', va='bottom', fontsize=20, color='black')

# Customize the plot
plt.xlabel('')
plt.ylabel('Exponent Value (a.u.)')
plt.title('1/f Exponent in Each Bout Across Sleep Stages')
plt.legend(title='Subject')
plt.grid(axis='y', alpha=0.3)

# Remove top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
## Compare different conditions for avg_exp value

# Filter data for light and dark periods
light_data = combined_df_zt[(combined_df_zt['ZT'] >= 0) & (combined_df_zt['ZT'] < 12)]
dark_data = combined_df_zt[(combined_df_zt['ZT'] >= 12) | (combined_df_zt['ZT'] < 0)]  # Assuming ZT=24 wraps around to ZT=0

# Calculate average exponent for each subject during light and dark periods
avg_exp_light = light_data.groupby('subject')['avg_exps'].mean()
avg_exp_dark = dark_data.groupby('subject')['avg_exps'].mean()

# Calculate mean and standard error for light and dark periods
mean_light = avg_exp_light.mean()
mean_dark = avg_exp_dark.mean()
sem_light = avg_exp_light.sem()
sem_dark = avg_exp_dark.sem()

# Calculate average exponent for each day in light and dark periods
light_avg_exps = light_data.groupby(light_data['Timestamp'].dt.date)['avg_exps'].mean()
dark_avg_exps = dark_data.groupby(dark_data['Timestamp'].dt.date)['avg_exps'].mean()

# Create DataFrame for swarmplot
data = pd.concat([light_avg_exps, dark_avg_exps], axis=0).reset_index(name='avg_exps')
data['Period'] = ['Light']*len(light_avg_exps) + ['Dark']*len(dark_avg_exps)

print(f"Light period: mean = {mean_light:.4f}, SEM = {sem_light:.4f}")
print(f"Dark period: mean = {mean_dark:.4f}, SEM = {sem_dark:.4f}")

# Perform t-test
t_stat, p_value = stats.ttest_ind(avg_exp_light, avg_exp_dark)

print(f"\nt-test results: t = {t_stat:.4f}, p = {p_value:.4f}")

# Set seaborn style
sns.set(style='white')

# Plot the data
plt.figure(figsize=(10, 10))
plt.errorbar(['Light', 'Dark'], [mean_light, mean_dark], yerr=[sem_light, sem_dark], 
             capsize=5, marker='o', color='black', linestyle='-', 
             markerfacecolor='lightskyblue', markeredgecolor='black')
sns.swarmplot(x='Period', y='avg_exps', data=data, palette=['orange', 'gray'], alpha=0.3)
plt.ylabel('Exponent Value (a.u.)')
plt.xlabel('')
plt.title('1/f Exponent Across Light and Dark Periods (plotting each day)')
plt.xlim(-0.2, 1.2)  # Reduce x-axis limits
plt.tight_layout()

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.gcf().set_size_inches(4, 4)  # Reduce figure size
plt.show()

In [ ]:
# Set seaborn style
sns.set(style='white')

# Plot the data
plt.figure(figsize=(10, 10))
plt.errorbar(['Light', 'Dark'], [mean_light, mean_dark], 
             capsize=5, marker='x', color='black', linestyle='None',
             markerfacecolor='lightskyblue', markeredgecolor='black')

# Plot connected dots for each subject
for i, (light_val, dark_val) in enumerate(zip(avg_exp_light.values, avg_exp_dark.values)):
    plt.plot(['Light', 'Dark'], [light_val, dark_val], color='black', alpha=0.5)
    plt.scatter('Light', light_val, color='orange', alpha=0.3)
    plt.scatter('Dark', dark_val, color='gray', alpha=0.3)

plt.ylabel('Exponent Value (a.u.)')
plt.xlabel('')
plt.title('1/f Exponent Across Light and Dark Periods (plotting each subject)')
plt.xlim(-0.2, 1.2)  # Reduce x-axis limits
plt.tight_layout()

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.gcf().set_size_inches(4, 4)  # Reduce figure size
plt.show()

In [ ]:
## Plot the change in exponent value across light and dark period when pooling all periods

# Define quartiles based on ZT
light_data['quartile'] = pd.cut(light_data['ZT'], bins=[0, 3, 6, 9, 12], 
                                labels=['Q1 (0-3)', 'Q2 (3-6)', 'Q3 (6-9)', 'Q4 (9-12)'], 
                                include_lowest=True)
dark_data['quartile'] = pd.cut(dark_data['ZT'], bins=[12, 15, 18, 21, 24], 
                               labels=['Q1 (12-15)', 'Q2 (15-18)', 'Q3 (18-21)', 'Q4 (21-24)'], 
                               include_lowest=True)

# Calculate average avg_exps in each quartile for each subject
light_avg_exps_quartiles = light_data.groupby(['subject', 'quartile'])['avg_exps'].mean()
dark_avg_exps_quartiles = dark_data.groupby(['subject', 'quartile'])['avg_exps'].mean()

# Reset index to prepare data for AnovaRM
light_rm_data = light_avg_exps_quartiles.reset_index()
dark_rm_data = dark_avg_exps_quartiles.reset_index()

# Perform repeated measures ANOVA

light_anova_rm = AnovaRM(data=light_rm_data, depvar='avg_exps', subject='subject', within=['quartile']).fit()
dark_anova_rm = AnovaRM(data=dark_rm_data, depvar='avg_exps', subject='subject', within=['quartile']).fit()

print("\nRepeated Measures ANOVA Results for light period:")
print(light_anova_rm.anova_table)
print("\nRepeated Measures ANOVA Results for dark period:")
print(dark_anova_rm.anova_table)

# Check for significant ANOVA and run post hoc pairwise comparisons
light_pairwise, dark_pairwise = None, None

if light_anova_rm.anova_table['Pr > F'][0] < 0.05:
    print("\nSignificant differences found in light period. Running pairwise comparisons...")
    light_pairwise = pg.pairwise_tests(
        data=light_rm_data,
        dv='avg_exps',
        within='quartile',
        subject='subject',
        padjust='bonf'
    )
    print(light_pairwise)

if dark_anova_rm.anova_table['Pr > F'][0] < 0.05:
    print("\nSignificant differences found in dark period. Running pairwise comparisons...")
    dark_pairwise = pg.pairwise_tests(
        data=dark_rm_data,
        dv='avg_exps',
        within='quartile',
        subject='subject',
        padjust='bonf'
    )
    print(dark_pairwise)

# Plot the data

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Light Period Plot
sns.lineplot(x='quartile', y='avg_exps', hue='subject', data=light_rm_data, alpha=0.3, ax=ax[0])
mean_light = light_rm_data.groupby('quartile')['avg_exps'].mean()
sem_light = light_rm_data.groupby('quartile')['avg_exps'].sem()
ax[0].errorbar(mean_light.index, mean_light, yerr=sem_light, fmt='o-', color='orange', capsize=5, linewidth=2.5)

# Annotate significant comparisons for light period
if light_pairwise is not None:
    for _, row in light_pairwise.iterrows():
        if row['p-corr'] < 0.05:
            ax[0].text(
                x=(int(row['A'][1]) + int(row['B'][1])) / 2 - 1,
                y=mean_light.max() + 0.03,
                s='*',
                ha='center',
                fontsize=12,
                color='red'
            )

ax[0].set_xlabel('Zeitgeber Quartile')
ax[0].set_ylabel('Exponent Value (a.u.)')
ax[0].set_title('1/f Exponent Across Quartiles - Light Period')
ax[0].set_ylim([1.05, 1.55])

# Dark Period Plot
sns.lineplot(x='quartile', y='avg_exps', hue='subject', data=dark_rm_data, alpha=0.3, ax=ax[1])
mean_dark = dark_rm_data.groupby('quartile')['avg_exps'].mean()
sem_dark = dark_rm_data.groupby('quartile')['avg_exps'].sem()
ax[1].errorbar(mean_dark.index, mean_dark, yerr=sem_dark, fmt='o-', color='gray', capsize=5, linewidth=2.5)

# Annotate significant comparisons for dark period
if dark_pairwise is not None:
    for _, row in dark_pairwise.iterrows():
        if row['p-corr'] < 0.05:
            ax[1].text(
                x=(int(row['A'][1]) + int(row['B'][1])) / 2 - 1,
                y=mean_dark.max() + 0.03,
                s='*',
                ha='center',
                fontsize=12,
                color='red'
            )

ax[1].set_xlabel('Zeitgeber Quartile')
ax[1].set_ylabel('Exponent Value (a.u.)')
ax[1].set_title('1/f Exponent Across Quartiles - Dark Period')
ax[1].set_ylim([1.05, 1.55])

# Remove top and right spines for both subplots
for a in ax:
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import pandas as pd
import numpy as np

# Define quartiles for the light period based on ZT
light_data['quartile'] = pd.cut(light_data['ZT'], bins=[0, 3, 6, 9, 12], 
                                labels=['Q1 (0-3)', 'Q2 (3-6)', 'Q3 (6-9)', 'Q4 (9-12)'], 
                                include_lowest=True)

# Separate light data into different sleep stages
light_wake = light_data[light_data['sleepStage'] == 1]
light_nrem = light_data[light_data['sleepStage'] == 2]
light_rem = light_data[light_data['sleepStage'] == 3]

# Function to prepare data for each sleep stage
def prepare_stage_data(stage_data):
    avg_exps_quartiles = stage_data.groupby(['subject', 'quartile'])['avg_exps'].mean()
    return avg_exps_quartiles.reset_index()

light_wake_data = prepare_stage_data(light_wake)
light_nrem_data = prepare_stage_data(light_nrem)
light_rem_data = prepare_stage_data(light_rem)

# Function to run repeated measures ANOVA and post-hoc tests
def run_anova_and_posthoc(stage_data, stage_name):
    # Perform repeated measures ANOVA
    aov = pg.rm_anova(dv='avg_exps', within='quartile', subject='subject', data=stage_data, detailed=True)
    
    print(f"\nRepeated Measures ANOVA Results for {stage_name}:")
    print(aov)
    
    posthoc_results = None
    if aov['p-unc'][0] < 0.05:
        print(f"\nPost-hoc pairwise comparisons for {stage_name}:")
        posthoc_results = pg.pairwise_ttests(dv='avg_exps', within='quartile', subject='subject', data=stage_data, padjust='bonferroni')
        print(posthoc_results)
    
    return aov, posthoc_results

# Run ANOVA and post-hoc tests
wake_anova, wake_posthoc = run_anova_and_posthoc(light_wake_data, 'Wake')
nrem_anova, nrem_posthoc = run_anova_and_posthoc(light_nrem_data, 'NREM')
rem_anova, rem_posthoc = run_anova_and_posthoc(light_rem_data, 'REM')

# Plot settings
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Function to plot stage data with significance annotations
def plot_stage_data(data, ax, title, color, posthoc_results):
    sns.lineplot(x='quartile', y='avg_exps', hue='subject', data=data, alpha=0.3, ax=ax, palette='Set1')
    mean_data = data.groupby('quartile')['avg_exps'].mean()
    sem_data = data.groupby('quartile')['avg_exps'].sem()
    ax.errorbar(mean_data.index, mean_data, yerr=sem_data, fmt='o-', color=color, capsize=5, linewidth=2.5, label='Mean')
    ax.set_title(title)
    ax.set_xlabel('Zeitgeber Quartile')
    ax.set_ylabel('Exponent Value (a.u.)')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add significance markers if posthoc results exist
    if posthoc_results is not None:
        # To track the y_max and avoid overlap of significance lines
        max_y = data['avg_exps'].max() + 0.05
        for _, row in posthoc_results.iterrows():
            if row['p-corr'] < 0.05:
                # Use the categorical quartile labels directly from post-hoc test
                quartile_1 = row['A']
                quartile_2 = row['B']
                
                # Use the index positions of the quartile labels for plotting
                quartile_1_idx = data[data['quartile'] == quartile_1].iloc[0].name
                quartile_2_idx = data[data['quartile'] == quartile_2].iloc[0].name

                # Define some vertical offset for the significance lines
                y_offset = 0.1  # Adjust the vertical distance between significance lines
                
                # Plot the significance line at y_max for the first quartile pair
                ax.plot([quartile_1_idx, quartile_2_idx], [max_y, max_y], color='black', linewidth=1.5)
                ax.text((quartile_1_idx + quartile_2_idx) / 2, max_y + 0.0005, '*', ha='center', va='bottom', color='black', fontsize=20)

                # Update max_y to be higher for the next significance line
                max_y += y_offset

# Plot for Wake
plot_stage_data(light_wake_data, axes[0], 'Wake - Light Phase', color='blue', posthoc_results=wake_posthoc)

# Plot for NREM
plot_stage_data(light_nrem_data, axes[1], 'NREM - Light Phase', color='green', posthoc_results=nrem_posthoc)

# Plot for REM
plot_stage_data(light_rem_data, axes[2], 'REM - Light Phase', color='red', posthoc_results=rem_posthoc)

# Remove top and right spines for all plots
for ax in axes:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Adjust layout
plt.tight_layout()
plt.show()



In [ ]:
## Plot the change in exponent value across different stage quartiles during dark phase

# Define quartiles for the dark period based on ZT
dark_data['quartile'] = pd.cut(dark_data['ZT'], bins=[12, 15, 18, 21, 24], 
                               labels=['Q1 (12-15)', 'Q2 (15-18)', 'Q3 (18-21)', 'Q4 (21-24)'], 
                               include_lowest=True)

# Separate dark data into different sleep stages
dark_wake = dark_data[dark_data['sleepStage'] == 1]
dark_nrem = dark_data[dark_data['sleepStage'] == 2]
dark_rem = dark_data[dark_data['sleepStage'] == 3]

# Function to prepare data for each sleep stage
def prepare_stage_data(stage_data):
    avg_exps_quartiles = stage_data.groupby(['subject', 'quartile'])['avg_exps'].mean()
    return avg_exps_quartiles.reset_index()

dark_wake_data = prepare_stage_data(dark_wake)
dark_nrem_data = prepare_stage_data(dark_nrem)
dark_rem_data = prepare_stage_data(dark_rem)

# Function to run repeated measures ANOVA and post-hoc tests
def run_anova_and_posthoc(stage_data, stage_name):
    # Perform repeated measures ANOVA
    aov = pg.rm_anova(dv='avg_exps', within='quartile', subject='subject', data=stage_data, detailed=True)
    
    print(f"\nRepeated Measures ANOVA Results for {stage_name}:")
    print(aov)
    
    posthoc_results = None
    if aov['p-unc'][0] < 0.05:
        print(f"\nPost-hoc pairwise comparisons for {stage_name}:")
        posthoc_results = pg.pairwise_ttests(dv='avg_exps', within='quartile', subject='subject', data=stage_data, padjust='bonferroni')
        print(posthoc_results)
    
    return aov, posthoc_results

# Run ANOVA and post-hoc tests for dark period
dark_wake_anova, dark_wake_posthoc = run_anova_and_posthoc(dark_wake_data, 'Wake')
dark_nrem_anova, dark_nrem_posthoc = run_anova_and_posthoc(dark_nrem_data, 'NREM')
dark_rem_anova, dark_rem_posthoc = run_anova_and_posthoc(dark_rem_data, 'REM')

# Mapping of quartile labels to numeric values for plotting significance
quartile_mapping = {'Q1 (12-15)': 1, 'Q2 (15-18)': 2, 'Q3 (18-21)': 3, 'Q4 (21-24)': 4}

# Plot settings
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Function to plot stage data with significance annotations
def plot_stage_data(data, ax, title, color, posthoc_results):
    sns.lineplot(x='quartile', y='avg_exps', hue='subject', data=data, alpha=0.3, ax=ax, palette='Set1')
    mean_data = data.groupby('quartile')['avg_exps'].mean()
    sem_data = data.groupby('quartile')['avg_exps'].sem()
    ax.errorbar(mean_data.index, mean_data, yerr=sem_data, fmt='o-', color=color, capsize=5, linewidth=2.5, label='Mean')
    ax.set_title(title)
    ax.set_xlabel('Zeitgeber Quartile')
    ax.set_ylabel('Exponent Value (a.u.)')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add significance markers if posthoc results exist
    if posthoc_results is not None:
        for _, row in posthoc_results.iterrows():
            if row['p-corr'] < 0.05:
                quartile_1 = quartile_mapping[row['A']]
                quartile_2 = quartile_mapping[row['B']]
                
                y_max = data['avg_exps'].max() + 0.05
                # Shift the asterisk downward by increasing the offset
                ax.plot([quartile_1, quartile_2], [y_max, y_max], color='black', linewidth=1.5)
                # Adjust the position of the asterisk further down
                ax.text((quartile_1 + quartile_2) / 2, y_max - 0.05, '*', ha='center', va='bottom', color='black')

# Plot for Wake in Dark period
plot_stage_data(dark_wake_data, axes[0], 'Wake - Dark Phase', color='blue', posthoc_results=dark_wake_posthoc)

# Plot for NREM in Dark period
plot_stage_data(dark_nrem_data, axes[1], 'NREM - Dark Phase', color='green', posthoc_results=dark_nrem_posthoc)

# Plot for REM in Dark period
plot_stage_data(dark_rem_data, axes[2], 'REM - Dark Phase', color='red', posthoc_results=dark_rem_posthoc)

# Remove top and right spines for all plots
for ax in axes:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
## Plot length of time awake vs exponent in first 10 seconds of NREM stage across all phases - run and plot linear regression

# Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Find rows immediately after each wake bout
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout
next_rows = combined_df_zt.loc[wake_bout_ends + 1]  # Rows immediately after wake bouts

# Combine wake bout length with the subsequent avg_exps value
wake_bout_analysis = pd.DataFrame({
    'wake_bout_length': wake_bouts['wake_bout_length'].values,
    'next_avg_exps': next_rows['avg_exps'].values
})

# Perform linear regression using scipy.stats.linregress
slope, intercept, r_value, p_value, std_err = linregress(wake_bout_analysis['wake_bout_length'], 
                                                         wake_bout_analysis['next_avg_exps'])

# Plot scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=wake_bout_analysis, x='wake_bout_length', y='next_avg_exps', s=50, alpha=0.7)

# Plot regression line
plt.plot(wake_bout_analysis['wake_bout_length'], intercept + slope * wake_bout_analysis['wake_bout_length'], color='orange')

# Add R-squared and p-value to the plot
plt.text(0.95, 0.05, f'R² = {r_value**2:.2f}\nP-value = {p_value:.2f}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right',
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

# Customize plot appearance
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in First NREM Epoch')
plt.legend()
plt.grid(alpha=0.3)

plt.show()


In [ ]:
## Plot length of time awake vs exponent in first 10 seconds of NREM stage across all phases splitting by light and dark - run and plot linear regression

# Function to analyze a given phase (light or dark)
def analyze_phase(data, phase_name):
    # Create a new column to identify wake bout groups
    data['wake_bout'] = (data['sleepStage'] != 1).cumsum()
    
    # Filter rows where sleepStage is wake (1)
    wake_data = data[data['sleepStage'] == 1]
    
    # Calculate the length of each wake bout
    wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')
    
    # Find rows immediately after each wake bout
    wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout
    next_indices = wake_bout_ends + 1  # Indices for the next rows
    valid_indices = next_indices[next_indices.isin(data.index)]  # Keep only valid indices
    
    next_rows = data.loc[valid_indices]  # Rows immediately after wake bouts
    
    # Combine wake bout length with the subsequent avg_exps value
    wake_bout_analysis = pd.DataFrame({
        'wake_bout_length': wake_bouts['wake_bout_length'].iloc[:len(next_rows)].values,
        'next_avg_exps': next_rows['avg_exps'].values
    })
    
    return wake_bout_analysis.dropna()

# Analyze light and dark phases
light_analysis = analyze_phase(light_data, 'Light')
dark_analysis = analyze_phase(dark_data, 'Dark')

# Function to perform linear regression and return results
def perform_regression(data):
    slope, intercept, r_value, p_value, std_err = linregress(data['wake_bout_length'], data['next_avg_exps'])
    r_squared = r_value**2
    return slope, intercept, r_squared, p_value

# Perform regression for light and dark phases
light_slope, light_intercept, light_r_squared, light_p_value = perform_regression(light_analysis)
dark_slope, dark_intercept, dark_r_squared, dark_p_value = perform_regression(dark_analysis)

# Predict values for the regression lines
light_analysis['predicted'] = light_intercept + light_slope * light_analysis['wake_bout_length']
dark_analysis['predicted'] = dark_intercept + dark_slope * dark_analysis['wake_bout_length']

# Plot the results
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Light phase subplot
sns.scatterplot(data=light_analysis, x='wake_bout_length', y='next_avg_exps', s=50, alpha=0.7, ax=axes[0], color='blue')
axes[0].plot(light_analysis['wake_bout_length'], light_analysis['predicted'], color='cyan')

# Annotate R² and p-value
axes[0].text(0.95, 0.05, f'R² = {light_r_squared:.2f}\nP-value = {light_p_value:.2f}',
             transform=axes[0].transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

axes[0].set_title('Wake Bout Length vs. 1/f NREM Exponent (Light Phase)')
axes[0].set_xlabel('Wake Bout Length (epochs)')
axes[0].set_ylabel('Exponent Value (a.u.)')
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].grid(alpha=0.3)
axes[0].legend()

# Dark phase subplot
sns.scatterplot(data=dark_analysis, x='wake_bout_length', y='next_avg_exps', s=50, alpha=0.7, ax=axes[1], color='orange')
axes[1].plot(dark_analysis['wake_bout_length'], dark_analysis['predicted'], color='red')

# Annotate R² and p-value
axes[1].text(0.95, 0.05, f'R² = {dark_r_squared:.2f}\nP-value = {dark_p_value:.2f}',
             transform=axes[1].transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

axes[1].set_title('Wake Bout Length vs. 1/f NREM Exponent (Dark Phase)')
axes[1].set_xlabel('Wake Bout Length (epochs)')
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].grid(alpha=0.3)
axes[1].legend()

# Adjust layout
plt.tight_layout()
plt.show()



In [ ]:
## Plot length of time awake vs exponent in first 10 seconds of NREM stage across all phases splitting by light and dark - run and plot linear regression (exclude <=2 bout lengths)

# Function to analyze a given phase (light or dark), excluding wake bouts <= 2
def analyze_phase(data, phase_name):
    # Create a new column to identify wake bout groups
    data['wake_bout'] = (data['sleepStage'] != 1).cumsum()
    
    # Filter rows where sleepStage is wake (1)
    wake_data = data[data['sleepStage'] == 1]
    
    # Calculate the length of each wake bout
    wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')
    
    # Exclude wake bouts with length <= 2
    valid_wake_bouts = wake_bouts[wake_bouts['wake_bout_length'] > 2]
    valid_wake_data = wake_data[wake_data['wake_bout'].isin(valid_wake_bouts['wake_bout'])]
    
    # Find rows immediately after each valid wake bout
    wake_bout_ends = valid_wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout
    next_indices = wake_bout_ends + 1  # Indices for the next rows
    valid_indices = next_indices[next_indices.isin(data.index)]  # Keep only valid indices
    
    next_rows = data.loc[valid_indices]  # Rows immediately after wake bouts
    
    # Combine wake bout length with the subsequent avg_exps value
    wake_bout_analysis = pd.DataFrame({
        'wake_bout_length': valid_wake_bouts['wake_bout_length'].iloc[:len(next_rows)].values,
        'next_avg_exps': next_rows['avg_exps'].values
    })
    
    return wake_bout_analysis.dropna()

# Analyze light and dark phases
light_analysis = analyze_phase(light_data, 'Light')
dark_analysis = analyze_phase(dark_data, 'Dark')

# Perform regression for light and dark phases
light_slope, light_intercept, light_r_squared, light_p_value = perform_regression(light_analysis)
dark_slope, dark_intercept, dark_r_squared, dark_p_value = perform_regression(dark_analysis)

# Predict values for the regression lines
light_analysis['predicted'] = light_intercept + light_slope * light_analysis['wake_bout_length']
dark_analysis['predicted'] = dark_intercept + dark_slope * dark_analysis['wake_bout_length']

# Plot the results
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Light phase subplot
sns.scatterplot(data=light_analysis, x='wake_bout_length', y='next_avg_exps', s=50, alpha=0.7, ax=axes[0], color='blue')
axes[0].plot(light_analysis['wake_bout_length'], light_analysis['predicted'], color='cyan')

# Annotate R² and p-value
axes[0].text(0.95, 0.05, f'R² = {light_r_squared:.2f}\nP-value = {light_p_value:.2f}',
             transform=axes[0].transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

axes[0].set_title('Wake Bout Length vs. 1/f NREM Exponent (Light Phase) Excluding <=2 Wake Bouts')
axes[0].set_xlabel('Wake Bout Length (epochs)')
axes[0].set_ylabel('Exponent Value (a.u.)')
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].grid(alpha=0.3)

# Dark phase subplot
sns.scatterplot(data=dark_analysis, x='wake_bout_length', y='next_avg_exps', s=50, alpha=0.7, ax=axes[1], color='orange')
axes[1].plot(dark_analysis['wake_bout_length'], dark_analysis['predicted'], color='red')

# Annotate R² and p-value
axes[1].text(0.95, 0.05, f'R² = {dark_r_squared:.2f}\nP-value = {dark_p_value:.2f}',
             transform=axes[1].transAxes, fontsize=12, verticalalignment='bottom', horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

axes[1].set_title('Wake Bout Length vs. 1/f NREM Exponent (Dark Phase) Excluding <=2 Wake Bouts')
axes[1].set_xlabel('Wake Bout Length (epochs)')
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].grid(alpha=0.3)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
## Plot length of time awake vs exponent across the length of the oroceeding NREM stage across all phases & run linear regression

# Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Find rows immediately after each wake bout (the start of the next NREM bout)
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout

# Initialize lists to hold analysis data
wake_bout_lengths = []
avg_exps_after_wake = []

# Iterate over each wake bout to find the subsequent NREM bout
for end_idx in wake_bout_ends:
    # Find the index of the first NREM epoch after the wake bout
    start_nrem_idx = end_idx + 1
    
    # Ensure we don't go out of bounds
    if start_nrem_idx >= len(combined_df_zt):
        continue
    
    # Find the rows where sleepStage == 2 (NREM) after the wake bout
    nrem_data = combined_df_zt.loc[start_nrem_idx:]
    nrem_data = nrem_data[nrem_data['sleepStage'] == 2]
    
    # Now we need to capture the whole NREM bout and stop when sleepStage is no longer 2
    nrem_bout_data = []
    for idx, row in nrem_data.iterrows():
        # If the sleepStage stays 2, keep adding the data to the bout
        nrem_bout_data.append(row)
        # If the next row isn't NREM, we stop the bout
        if idx + 1 < len(combined_df_zt) and combined_df_zt.loc[idx + 1, 'sleepStage'] != 2:
            break
    
    # If there was a NREM bout, calculate the average exponent value
    if nrem_bout_data:
        nrem_bout_df = pd.DataFrame(nrem_bout_data)
        avg_exp = nrem_bout_df['avg_exps'].mean()
        
        # Append the wake bout length and the average exponent for this NREM bout
        wake_bout_lengths.append(wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0])
        avg_exps_after_wake.append(avg_exp)

# Create a DataFrame for the analysis
wake_bout_analysis = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths,
    'avg_exp_after_wake': avg_exps_after_wake
})

# Drop rows where avg_exp_after_wake is NaN (e.g., no NREM bout follows the wake bout)
wake_bout_analysis = wake_bout_analysis.dropna()

# Perform linear regression analysis
slope, intercept, r_value, p_value, std_err = linregress(wake_bout_analysis['wake_bout_length'], wake_bout_analysis['avg_exp_after_wake'])

# Plot wake bout length vs. avg_exps in the following NREM period
plt.figure(figsize=(10, 6))
sns.scatterplot(data=wake_bout_analysis, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)

# Plot the regression line
plt.plot(wake_bout_analysis['wake_bout_length'], slope * wake_bout_analysis['wake_bout_length'] + intercept, color='orange')

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch')
plt.grid(alpha=0.3)

# Display the linear regression statistics on the plot
plt.legend()
plt.text(0.95, 0.05, f'R-squared = {r_value**2:.3f}\nSlope = {slope:.2f}\nP-value = {p_value:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

plt.show()


In [ ]:
## Plot length of time awake vs exponent across the length of the oroceeding NREM stage across all phases & run linear regression - ignore any bouts which are <= 2

# Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Find rows immediately after each wake bout (the start of the next NREM bout)
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout

# Initialize lists to hold analysis data
wake_bout_lengths = []
avg_exps_after_wake = []

# Iterate over each wake bout to find the subsequent NREM bout
for end_idx in wake_bout_ends:
    # Find the index of the first NREM epoch after the wake bout
    start_nrem_idx = end_idx + 1
    
    # Ensure we don't go out of bounds
    if start_nrem_idx >= len(combined_df_zt):
        continue
    
    # Find the rows where sleepStage == 2 (NREM) after the wake bout
    nrem_data = combined_df_zt.loc[start_nrem_idx:]
    nrem_data = nrem_data[nrem_data['sleepStage'] == 2]
    
    # Now we need to capture the whole NREM bout and stop when sleepStage is no longer 2
    nrem_bout_data = []
    for idx, row in nrem_data.iterrows():
        # If the sleepStage stays 2, keep adding the data to the bout
        nrem_bout_data.append(row)
        # If the next row isn't NREM, we stop the bout
        if idx + 1 < len(combined_df_zt) and combined_df_zt.loc[idx + 1, 'sleepStage'] != 2:
            break
    
    # If there was a NREM bout, calculate the average exponent value
    if nrem_bout_data:
        nrem_bout_df = pd.DataFrame(nrem_bout_data)
        avg_exp = nrem_bout_df['avg_exps'].mean()
        
        # Append the wake bout length and the average exponent for this NREM bout
        bout_length = wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0]
        
        # Only append if bout length is greater than 2
        if bout_length > 2:
            wake_bout_lengths.append(bout_length)
            avg_exps_after_wake.append(avg_exp)

# Create a DataFrame for the analysis
wake_bout_analysis = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths,
    'avg_exp_after_wake': avg_exps_after_wake
})

# Drop rows where avg_exp_after_wake is NaN (e.g., no NREM bout follows the wake bout)
wake_bout_analysis = wake_bout_analysis.dropna()

# Perform linear regression analysis
if len(wake_bout_analysis) > 1:  # Ensure there is enough data for regression
    slope, intercept, r_value, p_value, std_err = linregress(wake_bout_analysis['wake_bout_length'], wake_bout_analysis['avg_exp_after_wake'])

    # Plot wake bout length vs. avg_exps in the following NREM period
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=wake_bout_analysis, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)

    # Plot the regression line
    plt.plot(wake_bout_analysis['wake_bout_length'], slope * wake_bout_analysis['wake_bout_length'] + intercept, color='orange')

    # Remove top and right spines
    ax = plt.gca()  
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.xlabel('Wake Bout Length (epochs)')
    plt.ylabel('Exponent Value (a.u.)')
    plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch (exclude <=2 bout length)')
    plt.grid(alpha=0.3)

    # Display the linear regression statistics on the plot
    plt.legend()
    plt.text(0.95, 0.05, f'R-squared = {r_value**2:.3f}\nSlope = {slope:.2f}\nP-value = {p_value:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

    plt.show()
else:
    print("Not enough data for linear regression analysis after filtering out short wake bouts.")


In [ ]:
## Plot length of time awake vs exponent across the length of the oroceeding NREM stage across split across light and dark phase & run linear regression

# Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Find rows immediately after each wake bout (the start of the next NREM bout)
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout

# Initialize lists to hold analysis data
wake_bout_lengths_light = []
avg_exps_after_wake_light = []
wake_bout_lengths_dark = []
avg_exps_after_wake_dark = []

# Iterate over each wake bout to find the subsequent NREM bout
for end_idx in wake_bout_ends:
    # Find the index of the first NREM epoch after the wake bout
    start_nrem_idx = end_idx + 1
    
    # Ensure we don't go out of bounds
    if start_nrem_idx >= len(combined_df_zt):
        continue
    
    # Find the rows where sleepStage == 2 (NREM) after the wake bout
    nrem_data = combined_df_zt.loc[start_nrem_idx:]
    nrem_data = nrem_data[nrem_data['sleepStage'] == 2]
    
    # Now we need to capture the whole NREM bout and stop when sleepStage is no longer 2
    nrem_bout_data = []
    for idx, row in nrem_data.iterrows():
        # If the sleepStage stays 2, keep adding the data to the bout
        nrem_bout_data.append(row)
        # If the next row isn't NREM, we stop the bout
        if idx + 1 < len(combined_df_zt) and combined_df_zt.loc[idx + 1, 'sleepStage'] != 2:
            break
    
    # If there was a NREM bout, calculate the average exponent value
    if nrem_bout_data:
        nrem_bout_df = pd.DataFrame(nrem_bout_data)
        avg_exp = nrem_bout_df['avg_exps'].mean()
        
        # Determine whether it's light or dark phase
        zt = combined_df_zt.loc[end_idx, 'ZT']
        if 0 <= zt < 12:  # Light phase
            wake_bout_lengths_light.append(wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0])
            avg_exps_after_wake_light.append(avg_exp)
        else:  # Dark phase
            wake_bout_lengths_dark.append(wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0])
            avg_exps_after_wake_dark.append(avg_exp)

# Create DataFrames for the analysis
wake_bout_analysis_light = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths_light,
    'avg_exp_after_wake': avg_exps_after_wake_light
})

wake_bout_analysis_dark = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths_dark,
    'avg_exp_after_wake': avg_exps_after_wake_dark
})

# Drop rows where avg_exp_after_wake is NaN (e.g., no NREM bout follows the wake bout)
wake_bout_analysis_light = wake_bout_analysis_light.dropna()
wake_bout_analysis_dark = wake_bout_analysis_dark.dropna()

# Perform linear regression analysis
slope_light, intercept_light, r_value_light, p_value_light, std_err_light = linregress(wake_bout_analysis_light['wake_bout_length'], wake_bout_analysis_light['avg_exp_after_wake'])
slope_dark, intercept_dark, r_value_dark, p_value_dark, std_err_dark = linregress(wake_bout_analysis_dark['wake_bout_length'], wake_bout_analysis_dark['avg_exp_after_wake'])

# Plot wake bout length vs. avg_exps in the following NREM period for light and dark phases
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=wake_bout_analysis_light, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)
plt.plot(wake_bout_analysis_light['wake_bout_length'], slope_light * wake_bout_analysis_light['wake_bout_length'] + intercept_light, color='orange')

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch (Light)')
plt.grid(alpha=0.3)

# Display the linear regression statistics on the plot
plt.legend()
plt.text(0.95, 0.05, f'R-squared = {r_value_light**2:.3f}\nSlope = {slope_light:.2f}\nP-value = {p_value_light:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')


plt.subplot(1, 2, 2)
sns.scatterplot(data=wake_bout_analysis_dark, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)
plt.plot(wake_bout_analysis_dark['wake_bout_length'], slope_dark * wake_bout_analysis_dark['wake_bout_length'] + intercept_dark, color='orange')

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch (Dark)')
plt.grid(alpha=0.3)

# Display the linear regression statistics on the plot
plt.legend()
plt.text(0.95, 0.05, f'R-squared = {r_value_dark**2:.3f}\nSlope = {slope_dark:.2f}\nP-value = {p_value_dark:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Filter out wake bouts with length <= 2
wake_bouts = wake_bouts[wake_bouts['wake_bout_length'] > 2]

# Find rows immediately after each wake bout (the start of the next NREM bout)
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout

# Initialize lists to hold analysis data
wake_bout_lengths_light = []
avg_exps_after_wake_light = []
wake_bout_lengths_dark = []
avg_exps_after_wake_dark = []

# Iterate over each wake bout to find the subsequent NREM bout
for end_idx in wake_bout_ends:
    # Check if the wake bout length is > 2
    wake_bout_filter = wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout']]
    if wake_bout_filter.empty or wake_bout_filter['wake_bout_length'].values[0] <= 2:
        continue
    
    # Find the index of the first NREM epoch after the wake bout
    start_nrem_idx = end_idx + 1
    
    # Ensure we don't go out of bounds
    if start_nrem_idx >= len(combined_df_zt):
        continue
    
    # Find the rows where sleepStage == 2 (NREM) after the wake bout
    nrem_data = combined_df_zt.loc[start_nrem_idx:]
    nrem_data = nrem_data[nrem_data['sleepStage'] == 2]
    
    # Now we need to capture the whole NREM bout and stop when sleepStage is no longer 2
    nrem_bout_data = []
    for idx, row in nrem_data.iterrows():
        # If the sleepStage stays 2, keep adding the data to the bout
        nrem_bout_data.append(row)
        # If the next row isn't NREM, we stop the bout
        if idx + 1 < len(combined_df_zt) and combined_df_zt.loc[idx + 1, 'sleepStage'] != 2:
            break
    
    # If there was a NREM bout, calculate the average exponent value
    if nrem_bout_data:
        nrem_bout_df = pd.DataFrame(nrem_bout_data)
        avg_exp = nrem_bout_df['avg_exps'].mean()
        
        # Determine whether it's light or dark phase
        zt = combined_df_zt.loc[end_idx, 'ZT']
        if 0 <= zt < 12:  # Light phase
            wake_bout_lengths_light.append(wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0])
            avg_exps_after_wake_light.append(avg_exp)
        else:  # Dark phase
            wake_bout_lengths_dark.append(wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0])
            avg_exps_after_wake_dark.append(avg_exp)

# Create DataFrames for the analysis
wake_bout_analysis_light = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths_light,
    'avg_exp_after_wake': avg_exps_after_wake_light
})

wake_bout_analysis_dark = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths_dark,
    'avg_exp_after_wake': avg_exps_after_wake_dark
})

# Drop rows where avg_exp_after_wake is NaN (e.g., no NREM bout follows the wake bout)
wake_bout_analysis_light = wake_bout_analysis_light.dropna()
wake_bout_analysis_dark = wake_bout_analysis_dark.dropna()

# Perform linear regression analysis
slope_light, intercept_light, r_value_light, p_value_light, std_err_light = linregress(wake_bout_analysis_light['wake_bout_length'], wake_bout_analysis_light['avg_exp_after_wake'])
slope_dark, intercept_dark, r_value_dark, p_value_dark, std_err_dark = linregress(wake_bout_analysis_dark['wake_bout_length'], wake_bout_analysis_dark['avg_exp_after_wake'])

# Plot wake bout length vs. avg_exps in the following NREM period for light and dark phases
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=wake_bout_analysis_light, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)
plt.plot(wake_bout_analysis_light['wake_bout_length'], slope_light * wake_bout_analysis_light['wake_bout_length'] + intercept_light, color='orange')

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch (Light) exclude <=2 wake bout')
plt.grid(alpha=0.3)

# Display the linear regression statistics on the plot
plt.legend()
plt.text(0.95, 0.05, f'R-squared = {r_value_light**2:.3f}\nSlope = {slope_light:.2f}\nP-value = {p_value_light:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')


plt.subplot(1, 2, 2)
sns.scatterplot(data=wake_bout_analysis_dark, x='wake_bout_length', y='avg_exp_after_wake', s=50, alpha=0.7)
plt.plot(wake_bout_analysis_dark['wake_bout_length'], slope_dark * wake_bout_analysis_dark['wake_bout_length'] + intercept_dark, color='orange')

# Remove top and right spines
ax = plt.gca()  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Wake Bout Length (epochs)')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs. 1/f Exponent in NREM Epoch (Dark) exclude <=2 wake bout')
plt.grid(alpha=0.3)

# Display the linear regression statistics on the plot
plt.legend()
plt.text(0.95, 0.05, f'R-squared = {r_value_dark**2:.3f}\nSlope = {slope_dark:.2f}\nP-value = {p_value_dark:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Adjust layout to increase horizontal spacing between plots
plt.subplots_adjust(wspace=0.8)

plt.tight_layout()
plt.show()

In [ ]:
# Filter out wake_bout_length < 3
wake_data_filtered = wake_data[wake_data['wake_bout_length'] >= 3]

# Function to split bout into tertiles and distribute length appropriately
def split_into_tertiles(wake_bout_length):
    base_size = wake_bout_length // 3
    remainder = wake_bout_length % 3
    tertiles = [base_size + (1 if i < remainder else 0) for i in range(3)]
    return tertiles

# Function to assign tertiles within each bout_group
def assign_tertiles(group):
    tertiles = split_into_tertiles(group['wake_bout_length'].iloc[0])
    group['tert_group'] = np.concatenate([
        np.repeat(i + 1, tertiles[i]) for i in range(3)
    ])
    return group

# Apply tertile assignment within each bout_group
wake_data_filtered = wake_data_filtered.groupby('bout_group').apply(assign_tertiles)

# Calculate the average of avg_exps in each tertile
tertile_avg_exps = wake_data_filtered.groupby(['subject', 'tert_group'])['avg_exps'].mean().reset_index()

# Perform repeated measures ANOVA using Pingouin
anova_results = pg.rm_anova(dv='avg_exps', within='tert_group', subject='subject', data=tertile_avg_exps, detailed=True)

# Print the ANOVA results
print(anova_results)

# Check if the p-value is significant (less than 0.05)
p_value = anova_results['p-unc'][0]

# Plotting the results
plt.figure(figsize=(10, 6))

# Create a scatterplot for individual subjects
sns.scatterplot(data=tertile_avg_exps, x='tert_group', y='avg_exps', hue='subject', palette='Set1', legend=None, alpha=0.5, s=80)

# Plot the average across all subjects with error bars
sns.lineplot(data=tert_group_means, x='tert_group', y='avg_exps', color='black', lw=2)

# Calculate the means and standard errors for each tertile
tert_group_means = tertile_avg_exps.groupby('tert_group')['avg_exps'].mean().reset_index()
tert_group_sems = tertile_avg_exps.groupby('tert_group')['avg_exps'].sem().reset_index()

# Plot error bars (mean ± SEM) for each tertile
plt.errorbar(tert_group_means['tert_group'], tert_group_means['avg_exps'], 
             yerr=tert_group_sems['avg_exps'], fmt='none', color='black', lw=2, capsize=5)


# If the p-value is significant, perform post-hoc pairwise comparisons
if p_value < 0.05:
    print("Significant difference found, performing post-hoc pairwise comparisons...")
    
    # Perform post-hoc pairwise comparisons using Bonferroni correction
    post_hoc_results = pg.pairwise_ttests(dv='avg_exps', within='tert_group', subject='subject', data=tertile_avg_exps, padjust='bonf')
    
    # Print the results of the pairwise comparisons
    print(post_hoc_results)
    
    # Extract significant comparisons
    significant_comparisons = post_hoc_results[post_hoc_results['p-corrected'] < 0.05]
    
    # Plot significant pairwise comparisons with asterisks
    for _, row in significant_comparisons.iterrows():
        x1, x2 = row['A'], row['B']
        y = tertile_avg_exps['avg_exps'].max() + 0.05  # Adding a small offset for visibility
        plt.plot([x1, x2], [y, y], color='black', lw=1)
        plt.text((x1 + x2) / 2, y, '*', ha='center', va='bottom', color='black', fontsize=12)

else:
    print("No significant difference found.")

# Display grid and remove top/right spines for cleaner look
plt.grid(True, axis='y', alpha=0.3)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add labels and title
plt.title('1/f Exponent Across Tertiles of Wake Bouts')
plt.xlabel('Tertile of Wake Bout')
plt.ylabel('Exponent Value (a.u.)')
plt.xticks([1, 2, 3], labels=['1', '2', '3'])
plt.tight_layout()
plt.show()


In [ ]:
## During wake split each bout into tertiles and determine whether there is a change in the exponent value across the tertile - filter out bouts which have less than 3 length (create boxplots)

# Filter out wake_bout_length < 3
wake_data_filtered = wake_data[wake_data['wake_bout_length'] >= 3]

# Function to split bout into tertiles and distribute length appropriately
def split_into_tertiles(wake_bout_length):
    # Calculate the base size of each tertile
    base_size = wake_bout_length // 3
    remainder = wake_bout_length % 3
    
    # Distribute the remainder evenly across the tertiles
    tertiles = [base_size + (1 if i < remainder else 0) for i in range(3)]
    
    return tertiles

# Add a 'tert_group' column to indicate tertile number for each row in the bout
def assign_tertiles(group):
    tertiles = split_into_tertiles(group['wake_bout_length'].iloc[0])
    tert_start_idx = 0
    group['tert_group'] = np.concatenate([
        np.repeat(i + 1, tertiles[i]) for i in range(3)
    ])
    return group

# Apply tertile assignment within each bout_group
wake_data_filtered = wake_data_filtered.groupby('bout_group').apply(assign_tertiles)

# Now, we can directly plot all avg_exps for each subject in each tertile

# Plotting
plt.figure(figsize=(12, 6))

# Create a boxplot for each tertile
sns.boxplot(data=wake_data_filtered, x='tert_group', y='avg_exps', hue='tert_group', palette='Set1', 
            showmeans=True, meanline=True, linewidth=2)

# Calculate the average across all subjects for each tertile
tert_group_means = wake_data_filtered.groupby('tert_group')['avg_exps'].mean().reset_index()

# Add labels and title
plt.xlabel('Tertile of Wake Bout')
plt.ylabel('Exponent Value (a.u.)')
plt.title('1/f Exponent Across Tertiles of Wake Bouts')

# Customize legend
plt.legend('')

# Display grid and remove top/right spines for cleaner look
plt.grid(True, axis='y', alpha=0.3)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Tight layout for better spacing
plt.tight_layout()
plt.show()


In [ ]:
## Correlation of length of wake with its corresponding last tertile value in the same bout - exclude bouts <3 length

# Filter for tert_group = 3
tert_group_3 = wake_data_filtered[wake_data_filtered['tert_group'] == 3]

# Reset index to remove ambiguity between index and column labels
tert_group_3 = tert_group_3.reset_index(drop=True)

# Compute mean avg_exps for each bout_group when tert_group = 3
tert_group_3_means = tert_group_3.groupby('bout_group')['avg_exps'].mean().reset_index()
tert_group_3_means.rename(columns={'avg_exps': 'avg_exps_mean'}, inplace=True)

# Extract wake_bout_length for each bout_group
bout_lengths = wake_data_filtered[['bout_group', 'wake_bout_length']].drop_duplicates()

# Reset index for bout_lengths to avoid conflicts
bout_lengths = bout_lengths.reset_index(drop=True)

# Merge the mean avg_exps with bout_lengths
correlation_data = pd.merge(tert_group_3_means, bout_lengths, on='bout_group')

# Calculate correlation
correlation, p_value = stats.pearsonr(correlation_data['wake_bout_length'], correlation_data['avg_exps_mean'])

# Create scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(data=correlation_data, x='wake_bout_length', y='avg_exps_mean', color='blue', s=80, alpha=0.7)

# Add regression line
sns.regplot(data=correlation_data, x='wake_bout_length', y='avg_exps_mean', scatter=False, color='red', ci=None)

# Annotate with correlation results
plt.text(x=max(correlation_data['wake_bout_length']) * 0.7, 
         y=max(correlation_data['avg_exps_mean']) * 0.9,
         s=f"r = {correlation:.3f}\np = {p_value:.3f}",
         fontsize=12, color='black', bbox=dict(facecolor='white', edgecolor='gray', alpha=0.7))

# Customize plot
plt.xlabel('Wake Bout Length')
plt.ylabel('Exponent Value (a.u.)')
plt.title('Wake Bout Length vs 1/f Exponent in Tertile 3')
plt.grid(alpha=0.3)
plt.tight_layout()

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show plot
plt.show()


In [ ]:
## Correlation of length of waking period and the maximum exponent in the subsequent NREM episode - exclude bouts <=2 in length

# Step 1: Create a new column to identify wake bout groups
combined_df_zt['wake_bout'] = (combined_df_zt['sleepStage'] != 1).cumsum()

# Step 2: Filter rows where sleepStage is wake (1)
wake_data = combined_df_zt[combined_df_zt['sleepStage'] == 1]

# Step 3: Calculate the length of each wake bout
wake_bouts = wake_data.groupby('wake_bout').size().reset_index(name='wake_bout_length')

# Step 4: Find rows immediately after each wake bout (the start of the next NREM bout)
wake_bout_ends = wake_data.groupby('wake_bout').tail(1).index  # Last row of each wake bout

# Step 5: Initialize lists to hold analysis data
wake_bout_lengths = []
max_exps_after_wake = []

# Step 6: Iterate over each wake bout to find the subsequent NREM bout
for end_idx in wake_bout_ends:
    # Find the index of the first NREM epoch after the wake bout
    start_nrem_idx = end_idx + 1
    
    # Ensure we don't go out of bounds
    if start_nrem_idx >= len(combined_df_zt):
        continue
    
    # Find the rows where sleepStage == 2 (NREM) after the wake bout
    nrem_data = combined_df_zt.loc[start_nrem_idx:]
    nrem_data = nrem_data[nrem_data['sleepStage'] == 2]
    
    # Now we need to capture the whole NREM bout and stop when sleepStage is no longer 2
    nrem_bout_data = []
    for idx, row in nrem_data.iterrows():
        # If the sleepStage stays 2, keep adding the data to the bout
        nrem_bout_data.append(row)
        # If the next row isn't NREM, we stop the bout
        if idx + 1 < len(combined_df_zt) and combined_df_zt.loc[idx + 1, 'sleepStage'] != 2:
            break
    
    # If there was a NREM bout, calculate the maximum exponent value
    if nrem_bout_data:
        nrem_bout_df = pd.DataFrame(nrem_bout_data)
        max_exp = nrem_bout_df['avg_exps'].max()  # Get the maximum exponent value
        
        # Append the wake bout length and the maximum exponent for this NREM bout
        bout_length = wake_bouts.loc[wake_bouts['wake_bout'] == combined_df_zt.loc[end_idx, 'wake_bout'], 'wake_bout_length'].values[0]
        
        # Only append if bout length is greater than 2
        if bout_length > 2:
            wake_bout_lengths.append(bout_length)
            max_exps_after_wake.append(max_exp)

# Step 7: Create a DataFrame for the analysis
wake_bout_analysis = pd.DataFrame({
    'wake_bout_length': wake_bout_lengths,
    'max_exp_after_wake': max_exps_after_wake
})

# Step 8: Drop rows where max_exp_after_wake is NaN (e.g., no NREM bout follows the wake bout)
wake_bout_analysis = wake_bout_analysis.dropna()

# Step 9: Perform linear regression analysis
if len(wake_bout_analysis) > 1:  # Ensure there is enough data for regression
    slope, intercept, r_value, p_value, std_err = linregress(wake_bout_analysis['wake_bout_length'], wake_bout_analysis['max_exp_after_wake'])

    # Plot wake bout length vs. max_exps in the following NREM period
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=wake_bout_analysis, x='wake_bout_length', y='max_exp_after_wake', s=50, alpha=0.7)

    # Plot the regression line
    plt.plot(wake_bout_analysis['wake_bout_length'], slope * wake_bout_analysis['wake_bout_length'] + intercept, color='orange')

    # Remove top and right spines
    ax = plt.gca()  
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.xlabel('Wake Bout Length (epochs)')
    plt.ylabel('Max Exponent in Subsequent NREM Bout')
    plt.title('Wake Bout Length vs. Max Exponent in NREM Epoch (exclude <=2 bout length)')
    plt.grid(alpha=0.3)

    # Display the linear regression statistics on the plot
    plt.legend()
    plt.text(0.95, 0.05, f'R-squared = {r_value**2:.3f}\nSlope = {slope:.2f}\nP-value = {p_value:.3f}', transform=plt.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

    plt.show()
else:
    print("Not enough data for linear regression analysis after filtering out short wake bouts.")


In [ ]:
## During non-rem split each bout into tertiles and determine whether there is a change in the exponent value across the tertile - filter out bouts which have less than 3 length

# First add non-rem bout info to the combined_df_zt df

# Check if 'non_rem_bout_length' column already exists, and drop it if necessary
if 'non_rem_bout_length' in combined_df_zt.columns:
    combined_df_zt = combined_df_zt.drop(columns=['non_rem_bout_length'])

# Create 'non_rem_bout' column for identifying NREM bout group identifiers
combined_df_zt['non_rem_bout'] = (combined_df_zt['sleepStage'] != 2).cumsum()

# Filter for NREM epochs (sleepStage == 2)
nrem_data = combined_df_zt[combined_df_zt['sleepStage'] == 2]

# Calculate the length of each NREM bout (group size)
nrem_bout_lengths = nrem_data.groupby('non_rem_bout').size().reset_index(name='non_rem_bout_length')

# Merge the NREM bout length information and bout group back into combined_df_zt
combined_df_zt = combined_df_zt.merge(nrem_bout_lengths[['non_rem_bout', 'non_rem_bout_length']], 
                                       on='non_rem_bout', how='left')

# Filter out for non-rem data once it has been added to the combined_df_zt df
nrem_data = combined_df_zt[combined_df_zt['sleepStage'] == 2]

# Filter out NREM bouts with non_rem_bout_length < 3
nrem_data_filtered = nrem_data[nrem_data['non_rem_bout_length'] >= 3]

# Function to split NREM bout into tertiles and distribute length appropriately
def split_into_tertiles(nrem_bout_length):
    base_size = nrem_bout_length // 3
    remainder = nrem_bout_length % 3
    tertiles = [base_size + (1 if i < remainder else 0) for i in range(3)]
    return tertiles

def assign_tertiles(group):
    tertiles = split_into_tertiles(group['non_rem_bout_length'].iloc[0])
    group = group.copy()  # Avoid modifying the original DataFrame in-place
    group['tert_group'] = np.concatenate([
        np.repeat(i + 1, tertiles[i]) for i in range(3)
    ])
    return group

# Apply tertile assignment within each non-REM bout group
nrem_data_filtered = nrem_data_filtered.groupby('non_rem_bout').apply(assign_tertiles).reset_index(drop=True)

# Calculate the average of avg_exps in each tertile for NREM bouts
tertile_avg_exps_nrem = nrem_data_filtered.groupby(['subject', 'tert_group'])['avg_exps'].mean().reset_index()

# Calculate the average across all subjects for each tertile
tert_group_means_nrem = tertile_avg_exps_nrem.groupby('tert_group')['avg_exps'].mean().reset_index()

# Calculate standard error of the mean (SEM) for error bars
tertile_group_sems_nrem = tertile_avg_exps_nrem.groupby('tert_group')['avg_exps'].sem().reset_index()

# Perform repeated measures ANOVA using Pingouin
anova_results_nrem = pg.rm_anova(dv='avg_exps', within='tert_group', subject='subject', data=tertile_avg_exps_nrem, detailed=True)

# Print the ANOVA results
print("Repeated Measures ANOVA Results:\n", anova_results_nrem)

# Perform posthoc pairwise t-tests with correction for multiple comparisons
posthoc_results = pg.pairwise_ttests(
    dv='avg_exps', within='tert_group', subject='subject', 
    data=tertile_avg_exps_nrem, padjust='bonferroni'
)

# Print the posthoc test results
print("\nPosthoc Pairwise Comparisons (Bonferroni-corrected):\n", posthoc_results)

# Plotting
plt.figure(figsize=(12, 8))

# Scatterplot for individual subjects
scatter = sns.scatterplot(
    data=tertile_avg_exps_nrem, 
    x='tert_group', y='avg_exps', 
    hue='subject', palette='Set1', alpha=0.5, s=100
)

# Line plot linking the means across tertiles
sns.lineplot(
    data=tert_group_means_nrem, x='tert_group', y='avg_exps', 
    color='black', lw=2, label='Mean ± SEM'
)

# Add mean and SEM error bars
plt.errorbar(
    x=tert_group_means_nrem['tert_group'], 
    y=tert_group_means_nrem['avg_exps'], 
    yerr=tertile_group_sems_nrem['avg_exps'], 
    fmt='o', color='black', lw=2, capsize=5
)

# Add significant differences with bars lifted above the plot
y_offset = tert_group_means_nrem['avg_exps'].max() + 0.2  # Start higher above the data
for i, row in posthoc_results.iterrows():
    if row['p-corr'] < 0.05:  # Only annotate significant comparisons
        x1, x2 = row['A'], row['B']
        y = y_offset + i * 0.05  # Add spacing for multiple bars
        plt.plot([x1, x2], [y, y], color='black', lw=1.5)

        # Determine significance level and corresponding asterisks
        if row['p-corr'] < 0.001:
            significance = '***'
        elif row['p-corr'] < 0.01:
            significance = '**'
        elif row['p-corr'] < 0.05:
            significance = '*'
        else:
            significance = ''  # No marker for non-significant results

        # Annotate plot
        plt.text((x1 + x2) / 2, y + 0.0005, significance, ha='center', va='bottom', 
                 color='black', fontsize=20)

# Adjusting legend to show individual subject colors
handles, labels = scatter.get_legend_handles_labels()
plt.legend(
    handles=handles[:-1], labels=labels[:-1],  # Skip the 'subject' title entry
    title='Subjects', loc='upper left', borderaxespad=0
)

# Add labels, title, and grid
plt.title('1/f Exponent Across Tertiles of NREM Bouts')
plt.xlabel('Tertile of NREM Bout')
plt.ylabel('Exponent Value (a.u.)')
plt.xticks([1, 2, 3], labels=['1', '2', '3'])
plt.grid(True, axis='y', alpha=0.3)

# Clean up plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
nrem_data = nrem_data.drop(columns=['non_rem_bout_length_x', 'non_rem_bout_length_y'])

In [ ]:
## Does change of exponent across a NREM bout correlate with bout length - does longer bout length correlate with difference between first and last tertile

# Step 1: Filter for tert_group=1 and tert_group=3
tertile_groups = nrem_data_filtered[nrem_data_filtered['tert_group'].isin([1, 3])]

# Step 2: Calculate mean avg_exps for tert_group=1 and tert_group=3 within each bout
tertile_means = (
    tertile_groups.groupby(['non_rem_bout', 'tert_group'])['avg_exps']
    .mean()
    .unstack(level='tert_group')
)

# Ensure both tert_group=1 and tert_group=3 exist for each bout
tertile_means = tertile_means.dropna(subset=[1, 3])

# Step 3: Compute the difference between tert_group=1 and tert_group=3
tertile_means['mean_diff'] = tertile_means[3] - tertile_means[1]

# Step 4: Merge mean_diff with non_rem_bout_length
bout_lengths = nrem_data_filtered[['non_rem_bout', 'non_rem_bout_length']].drop_duplicates()
data_for_regression = pd.merge(
    bout_lengths, 
    tertile_means[['mean_diff']], 
    on='non_rem_bout'
)

# Step 5: Perform linear regression using linregress
x = data_for_regression['non_rem_bout_length']  # Independent variable
y = data_for_regression['mean_diff']  # Dependent variable

slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Print regression results
print("Linear Regression Results:")
print(f"Slope: {slope:.4f}")
print(f"Intercept: {intercept:.4f}")
print(f"R-squared: {r_value**2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Standard Error: {std_err:.4f}")

# Scatter plot of data points
plt.figure(figsize=(10, 6))
sns.scatterplot(x=x, y=y, color="blue", s=100, alpha=0.5)

# Plot regression line
regression_line = intercept + slope * x
plt.plot(x, regression_line, color="red", linewidth=2)

# Add R-squared and p-value annotations
plt.text(
    0.95, 0.95, 
    f"$R^2$ = {r_value**2:.2f}\nP-value = {p_value:.2f}", 
    transform=plt.gca().transAxes, fontsize=12, verticalalignment="top", color="black"
)

# Customize plot aesthetics
plt.title("NREM Bout Length vs Last to First Tertile 1/f Exponent Difference", fontsize=18)
plt.xlabel("Non-REM Bout Length", fontsize=12)
plt.ylabel("Exponent Difference (a.u.)", fontsize=12)
plt.grid(alpha=0.3)
plt.legend(fontsize=12)

# Clean up plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
## Determine whether there is a difference in mean exponents in NREM bouts across 3 hour ZT blocks

# Create ZT block column (e.g., ZT 0-3, 3-6, etc.)
nrem_data_filtered['zt_block'] = nrem_data_filtered['ZT'] // 3

# Group by subject and ZT block, calculate the mean avg_exps
zt_block_means = (
    nrem_data_filtered.groupby(['subject', 'zt_block'])['avg_exps']
    .mean()
    .reset_index()
)

# Format ZT block intervals for clarity
zt_block_means['zt_block_interval'] = zt_block_means['zt_block'].apply(lambda x: f"ZT {x*3}-{(x+1)*3}")

# Display results
print(zt_block_means)


In [ ]:
# Define custom x-tick labels for ZT intervals
zt_block_labels = [
    "0-3", "3-6", "6-9", "9-12", "12-15", "15-18", "18-21", "21-24"
]

# Calculate the mean and SEM for each ZT block interval across subjects
zt_block_summary = (
    zt_block_means.groupby('zt_block')['avg_exps']
    .agg(['mean', 'sem'])
    .reset_index()
    .rename(columns={'mean': 'mean_avg_exps', 'sem': 'sem_avg_exps'})
)

# Plotting
plt.figure(figsize=(12, 8))

# Scatterplot for individual subjects
scatter = sns.scatterplot(
    data=zt_block_means,
    x='zt_block',
    y='avg_exps',
    hue='subject',
    palette='Set1',
    alpha=0.6,
    s=100,
)

# Line plot with mean and SEM error bars
plt.errorbar(
    x=zt_block_summary['zt_block'],
    y=zt_block_summary['mean_avg_exps'],
    fmt='-o',
    color='black',
    lw=2,
    capsize=5,
)

# Add plot details
plt.title('NREM Bout 1/f Exponent Across 3-Hour ZT Blocks', fontsize=22)
plt.xlabel('ZT Interval', fontsize=14)
plt.ylabel('Exponent Value (a.u.)', fontsize=14)
plt.xticks(
    ticks=zt_block_summary['zt_block'], 
    labels=zt_block_labels, 
    rotation=0, 
    fontsize=12
)
plt.yticks(fontsize=12)
plt.legend(title='Subjects', fontsize=12)
plt.grid(axis='y', alpha=0.3)

# Adjust legend
plt.legend(title='Subjects', fontsize=12, loc='upper left')

# Clean up plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

# Show plot
plt.show()

# Perform repeated measures ANOVA
aov = pg.rm_anova(
    data=zt_block_means,
    dv='avg_exps',          # Dependent variable
    within='zt_block',      # Within-subject factor
    subject='subject',      # Subject identifier
    detailed=True           # Print detailed results
)
print("Repeated Measures ANOVA Results:")
print(aov)

# Perform pairwise comparisons with Bonferroni correction
posthoc = pg.pairwise_ttests(
    data=zt_block_means,
    dv='avg_exps',           # Dependent variable
    within='zt_block',       # Within-subject factor
    subject='subject',       # Subject identifier
    padjust='bonf'           # Bonferroni correction
)
print("\nPost Hoc Pairwise Comparisons with Bonferroni Correction:")
print(posthoc)
